In [1]:
#here we will get the stance and emotion label 
#then we cross validate with NRC lexicon for emotion
#then we cross validate with the FNC-1 dataset trained on a transformer-based model

import pandas as pd
import os

In [2]:
from tqdm import tqdm
tqdm.pandas()

In [3]:
os.chdir(r"D:\PACK\PhD\Study1\cleaned_dataset")


In [2]:
from nltk.tokenize import sent_tokenize

In [5]:
train_df = pd.read_pickle('train_NER.h5',compression='xz').drop_duplicates()
val_df = pd.read_pickle('val_NER.h5',compression='xz').drop_duplicates()
test_df = pd.read_pickle('test_NER.h5',compression='xz').drop_duplicates()

In [6]:
train_df.head(20)

,index,text,label,target,target_label
0,2812,California has had a movement going for the la...,0,California,GPE
0,2812,California has had a movement going for the la...,0,Donald Trump,PERSON
0,2812,California has had a movement going for the la...,0,the United States,GPE
0,2812,California has had a movement going for the la...,0,Trump,PERSON
0,2812,California has had a movement going for the la...,0,America,GPE
0,2812,California has had a movement going for the la...,0,Californians,NORP
0,2812,California has had a movement going for the la...,0,Pence,PERSON
0,2812,California has had a movement going for the la...,0,the GOP,ORG
0,2812,California has had a movement going for the la...,0,Congress,ORG
0,2812,California has had a movement going for the la...,0,the United States Constitution,ORG


In [7]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None,padding=True, truncation=True)

In [8]:
train_df.text.iloc[0]

'California has had a movement going for the last couple of  years that is pushing for the large, deep blue state to leave the union. This state has long been the leader of progressive policies and movements, and with the election of Donald Trump and his supporting regressive federal government, it is looking more and more like California just might be fed up enough to leave the United States permanently.The movement is led by the Yes California Independence campaign, and they have been given the green light to begin collecting signatures to get on the ballot. Thanks to the fact that the Trump administration wants to take America back to the dark ages where people of color are more disenfranchised than ever, where LGBTQ people are subjected to abuses like conversion therapy and routinely discriminated against, where women have no reproductive rights   well, let s just say the idea just got a whole hell of a lot less far-fetched. It s not a stretch to believe that the majority of Califo

In [9]:
#Method to try. Extract all sentences with the target and get the emotion and stance.
#My assumption is that it should not fluctuate  


In [10]:
lst = []
for i in tqdm(range(len(train_df))):
    text = train_df.text.iloc[i] 
    target = train_df.target.iloc[i]

    sentences = sent_tokenize(text)
    #print((len(sentences)))
    #print(target)
    _lst = [sentence for sentence in sentences if target in sentence]
    lst.append(_lst)
#df2 = pd.Series(lst)
train_df['sentences'] = lst
train_df['joined_sentences'] = train_df['sentences'].apply(lambda x: ' '.join(x))
train_df

100%|████████████████████████████████████████████████████████████████████████| 732131/732131 [10:36<00:00, 1150.34it/s]


,index,text,label,target,target_label,sentences,joined_sentences
0,2812,California has had a movement going for the la...,0,California,GPE,[California has had a movement going for the l...,California has had a movement going for the la...
0,2812,California has had a movement going for the la...,0,Donald Trump,PERSON,[This state has long been the leader of progre...,This state has long been the leader of progres...
0,2812,California has had a movement going for the la...,0,the United States,GPE,[This state has long been the leader of progre...,This state has long been the leader of progres...
0,2812,California has had a movement going for the la...,0,Trump,PERSON,[This state has long been the leader of progre...,This state has long been the leader of progres...
0,2812,California has had a movement going for the la...,0,America,GPE,[Thanks to the fact that the Trump administrat...,Thanks to the fact that the Trump administrati...
...,...,...,...,...,...,...,...
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Illinois Lottery,ORG,[The state will be suspended from participatin...,The state will be suspended from participating...
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Jason Schaumburg,PERSON,[The state will be suspended from participatin...,The state will be suspended from participating...
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Gianna Urgo,PERSON,"[With no money to pay road contractors, Illino...","With no money to pay road contractors, Illinoi..."
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Medicaid,ORG,[A U.S. judge has set a June 20 deadline for I...,A U.S. judge has set a June 20 deadline for Il...


In [11]:
lst = []
for i in tqdm(range(len(val_df))):
    text = val_df.text.iloc[i] 
    target = val_df.target.iloc[i]

    sentences = sent_tokenize(text)
    #print((len(sentences)))
    #print(target)
    _lst = [sentence for sentence in sentences if target in sentence]
    lst.append(_lst)
#df2 = pd.Series(lst)
val_df['sentences'] = lst
val_df['joined_sentences'] = val_df['sentences'].apply(lambda x: ' '.join(x))
val_df

100%|████████████████████████████████████████████████████████████████████████| 182084/182084 [02:45<00:00, 1102.47it/s]


,index,text,label,target,target_label,sentences,joined_sentences
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,LOS ANGELES,GPE,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,Reuters,ORG,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,Jennifer Holliday,PERSON,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,Donald Trump,PERSON,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,lesbian,NORP,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...
...,...,...,...,...,...,...,...
10125,5616,"KAUNUS, Lithuania (Reuters) - German Defense M...",1,Germany,GPE,[Mattis has sought to reassure NATO allies lik...,Mattis has sought to reassure NATO allies like...
10125,5616,"KAUNUS, Lithuania (Reuters) - German Defense M...",1,the United States,GPE,[Mattis has sought to reassure NATO allies lik...,Mattis has sought to reassure NATO allies like...
10125,5616,"KAUNUS, Lithuania (Reuters) - German Defense M...",1,Donald Trump,PERSON,[Mattis has sought to reassure NATO allies lik...,Mattis has sought to reassure NATO allies like...
10125,5616,"KAUNUS, Lithuania (Reuters) - German Defense M...",1,Europe,LOC,[Mattis has sought to reassure NATO allies lik...,Mattis has sought to reassure NATO allies like...


In [12]:
lst = []
for i in tqdm(range(len(test_df))):
    text = test_df.text.iloc[i] 
    target = test_df.target.iloc[i]

    sentences = sent_tokenize(text)
    #print((len(sentences)))
    #print(target)
    _lst = [sentence for sentence in sentences if target in sentence]
    lst.append(_lst)
#df2 = pd.Series(lst)
test_df['sentences'] = lst
test_df['joined_sentences'] = test_df['sentences'].apply(lambda x: ' '.join(x))
test_df

100%|████████████████████████████████████████████████████████████████████████| 226180/226180 [02:53<00:00, 1302.91it/s]


,index,text,label,target,target_label,sentences,joined_sentences
0,0,Message in grain field in N Dakota\n Believe i...,1,N Dakota,GPE,[Message in grain field in N Dakota\n Believe ...,Message in grain field in N Dakota\n Believe i...
0,0,Message in grain field in N Dakota\n Believe i...,1,Wells County,GPE,[The guy who tilled this message in his soil i...,The guy who tilled this message in his soil is...
0,0,Message in grain field in N Dakota\n Believe i...,1,Speedwell Township,GPE,"[9, 146:90 Speedwell Township, Wells County in...","9, 146:90 Speedwell Township, Wells County in ..."
0,0,Message in grain field in N Dakota\n Believe i...,1,North Dakota,GPE,"[9, 146:90 Speedwell Township, Wells County in...","9, 146:90 Speedwell Township, Wells County in ..."
1,1,Best Buy has announced they will be using “Hap...,1,Merry Christmas,WORK_OF_ART,[Best Buy has announced they will be using “Ha...,Best Buy has announced they will be using “Hap...
...,...,...,...,...,...,...,...
11342,1264,"In the early 1980s, Sen. Edward Kennedy secret...",0,Edward Kennedy,PERSON,"[In the early 1980s, Sen. Edward Kennedy secre...","In the early 1980s, Sen. Edward Kennedy secret..."
11342,1264,"In the early 1980s, Sen. Edward Kennedy secret...",0,Soviet,NORP,"[In the early 1980s, Sen. Edward Kennedy secre...","In the early 1980s, Sen. Edward Kennedy secret..."
11342,1264,"In the early 1980s, Sen. Edward Kennedy secret...",0,Reagan,PERSON,"[In the early 1980s, Sen. Edward Kennedy secre...","In the early 1980s, Sen. Edward Kennedy secret..."
11343,1265,Says an EPA permit languished under Strickland...,0,EPA,ORG,[Says an EPA permit languished under Stricklan...,Says an EPA permit languished under Strickland...


In [13]:
train_df['emotion'] = train_df.joined_sentences.progress_apply(lambda x: classifier(x))
val_df['emotion'] = val_df.joined_sentences.progress_apply(lambda x: classifier(x))
test_df['emotion'] = test_df.joined_sentences.progress_apply(lambda x: classifier(x))


100%|████████████████████████████████████████████████████████████████████████| 226180/226180 [5:35:47<00:00, 11.23it/s]


In [21]:
#Time Run: 17:41:14 + 4:21:08 + 5:35:47
#

In [14]:
train_df

,index,text,label,target,target_label,sentences,joined_sentences,emotion
0,2812,California has had a movement going for the la...,0,California,GPE,[California has had a movement going for the l...,California has had a movement going for the la...,"[[{'label': 'neutral', 'score': 0.605741798877..."
0,2812,California has had a movement going for the la...,0,Donald Trump,PERSON,[This state has long been the leader of progre...,This state has long been the leader of progres...,"[[{'label': 'neutral', 'score': 0.946747601032..."
0,2812,California has had a movement going for the la...,0,the United States,GPE,[This state has long been the leader of progre...,This state has long been the leader of progres...,"[[{'label': 'neutral', 'score': 0.720751821994..."
0,2812,California has had a movement going for the la...,0,Trump,PERSON,[This state has long been the leader of progre...,This state has long been the leader of progres...,"[[{'label': 'neutral', 'score': 0.536643862724..."
0,2812,California has had a movement going for the la...,0,America,GPE,[Thanks to the fact that the Trump administrat...,Thanks to the fact that the Trump administrati...,"[[{'label': 'surprise', 'score': 0.88948446512..."
...,...,...,...,...,...,...,...,...
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Illinois Lottery,ORG,[The state will be suspended from participatin...,The state will be suspended from participating...,"[[{'label': 'sadness', 'score': 0.664979696273..."
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Jason Schaumburg,PERSON,[The state will be suspended from participatin...,The state will be suspended from participating...,"[[{'label': 'sadness', 'score': 0.664979696273..."
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Gianna Urgo,PERSON,"[With no money to pay road contractors, Illino...","With no money to pay road contractors, Illinoi...","[[{'label': 'sadness', 'score': 0.831223368644..."
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Medicaid,ORG,[A U.S. judge has set a June 20 deadline for I...,A U.S. judge has set a June 20 deadline for Il...,"[[{'label': 'neutral', 'score': 0.787567675113..."


In [16]:
val_df

,index,text,label,target,target_label,sentences,joined_sentences,emotion
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,LOS ANGELES,GPE,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...,"[[{'label': 'sadness', 'score': 0.502611398696..."
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,Reuters,ORG,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...,"[[{'label': 'sadness', 'score': 0.968712568283..."
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,Jennifer Holliday,PERSON,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...,"[[{'label': 'sadness', 'score': 0.502611398696..."
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,Donald Trump,PERSON,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...,"[[{'label': 'disgust', 'score': 0.445870488882..."
0,6318,LOS ANGELES (Reuters) - Broadway star Jennifer...,1,lesbian,NORP,[LOS ANGELES (Reuters) - Broadway star Jennife...,LOS ANGELES (Reuters) - Broadway star Jennifer...,"[[{'label': 'disgust', 'score': 0.665051937103..."
...,...,...,...,...,...,...,...,...
10125,5616,"KAUNUS, Lithuania (Reuters) - German Defense M...",1,Germany,GPE,[Mattis has sought to reassure NATO allies lik...,Mattis has sought to reassure NATO allies like...,"[[{'label': 'neutral', 'score': 0.437396287918..."
10125,5616,"KAUNUS, Lithuania (Reuters) - German Defense M...",1,the United States,GPE,[Mattis has sought to reassure NATO allies lik...,Mattis has sought to reassure NATO allies like...,"[[{'label': 'neutral', 'score': 0.437396287918..."
10125,5616,"KAUNUS, Lithuania (Reuters) - German Defense M...",1,Donald Trump,PERSON,[Mattis has sought to reassure NATO allies lik...,Mattis has sought to reassure NATO allies like...,"[[{'label': 'neutral', 'score': 0.437396287918..."
10125,5616,"KAUNUS, Lithuania (Reuters) - German Defense M...",1,Europe,LOC,[Mattis has sought to reassure NATO allies lik...,Mattis has sought to reassure NATO allies like...,"[[{'label': 'neutral', 'score': 0.437396287918..."


In [17]:
test_df

,index,text,label,target,target_label,sentences,joined_sentences,emotion
0,0,Message in grain field in N Dakota\n Believe i...,1,N Dakota,GPE,[Message in grain field in N Dakota\n Believe ...,Message in grain field in N Dakota\n Believe i...,"[[{'label': 'neutral', 'score': 0.798286616802..."
0,0,Message in grain field in N Dakota\n Believe i...,1,Wells County,GPE,[The guy who tilled this message in his soil i...,The guy who tilled this message in his soil is...,"[[{'label': 'neutral', 'score': 0.413211166858..."
0,0,Message in grain field in N Dakota\n Believe i...,1,Speedwell Township,GPE,"[9, 146:90 Speedwell Township, Wells County in...","9, 146:90 Speedwell Township, Wells County in ...","[[{'label': 'neutral', 'score': 0.850890159606..."
0,0,Message in grain field in N Dakota\n Believe i...,1,North Dakota,GPE,"[9, 146:90 Speedwell Township, Wells County in...","9, 146:90 Speedwell Township, Wells County in ...","[[{'label': 'neutral', 'score': 0.850890159606..."
1,1,Best Buy has announced they will be using “Hap...,1,Merry Christmas,WORK_OF_ART,[Best Buy has announced they will be using “Ha...,Best Buy has announced they will be using “Hap...,"[[{'label': 'neutral', 'score': 0.923591732978..."
...,...,...,...,...,...,...,...,...
11342,1264,"In the early 1980s, Sen. Edward Kennedy secret...",0,Edward Kennedy,PERSON,"[In the early 1980s, Sen. Edward Kennedy secre...","In the early 1980s, Sen. Edward Kennedy secret...","[[{'label': 'disgust', 'score': 0.668400585651..."
11342,1264,"In the early 1980s, Sen. Edward Kennedy secret...",0,Soviet,NORP,"[In the early 1980s, Sen. Edward Kennedy secre...","In the early 1980s, Sen. Edward Kennedy secret...","[[{'label': 'disgust', 'score': 0.668400585651..."
11342,1264,"In the early 1980s, Sen. Edward Kennedy secret...",0,Reagan,PERSON,"[In the early 1980s, Sen. Edward Kennedy secre...","In the early 1980s, Sen. Edward Kennedy secret...","[[{'label': 'disgust', 'score': 0.668400585651..."
11343,1265,Says an EPA permit languished under Strickland...,0,EPA,ORG,[Says an EPA permit languished under Stricklan...,Says an EPA permit languished under Strickland...,"[[{'label': 'neutral', 'score': 0.703949332237..."


In [15]:
train_df.to_pickle('train_NER_emotion.h5')
val_df.to_pickle('val_NER_emotion.h5')
test_df.to_pickle('test_NER_emotion.h5')

In [19]:
stance_classifier = pipeline("text-classification", model="kornosk/bert-election2020-twitter-stance-trump", top_k=None,padding=True, truncation=True)


In [20]:
train_df['stance'] = train_df.joined_sentences.progress_apply(lambda x: stance_classifier(x))
train_df.to_pickle('train_NER_emotion_stance.h5')


val_df['stance'] = val_df.joined_sentences.progress_apply(lambda x: stance_classifier(x))
val_df.to_pickle('val_NER_emotion_stance.h5')

test_df['stance'] = test_df.joined_sentences.progress_apply(lambda x: stance_classifier(x))
test_df.to_pickle('test_NER_emotion_stance.h5')

100%|███████████████████████████████████████████████████████████████████████| 226180/226180 [11:11:42<00:00,  5.61it/s]


In [ ]:
'''
Runtimes
37:03:06 + 8:36:08 + 11:11:42

'''

In [4]:
#NRC
from nrclex import NRCLex


In [6]:
train_df = pd.read_pickle('train_NER_emotion_stance.h5')
val_df = pd.read_pickle('val_NER_emotion_stance.h5')
test_df = pd.read_pickle('test_NER_emotion_stance.h5')

train_df

,index,text,label,target,target_label,sentences,joined_sentences,emotion,stance
0,2812,California has had a movement going for the la...,0,California,GPE,[California has had a movement going for the l...,California has had a movement going for the la...,"[[{'label': 'neutral', 'score': 0.605741798877...","[[{'label': 'LABEL_0', 'score': 0.787573456764..."
0,2812,California has had a movement going for the la...,0,Donald Trump,PERSON,[This state has long been the leader of progre...,This state has long been the leader of progres...,"[[{'label': 'neutral', 'score': 0.946747601032...","[[{'label': 'LABEL_0', 'score': 0.709447622299..."
0,2812,California has had a movement going for the la...,0,the United States,GPE,[This state has long been the leader of progre...,This state has long been the leader of progres...,"[[{'label': 'neutral', 'score': 0.720751821994...","[[{'label': 'LABEL_0', 'score': 0.643858253955..."
0,2812,California has had a movement going for the la...,0,Trump,PERSON,[This state has long been the leader of progre...,This state has long been the leader of progres...,"[[{'label': 'neutral', 'score': 0.536643862724...","[[{'label': 'LABEL_0', 'score': 0.821133673191..."
0,2812,California has had a movement going for the la...,0,America,GPE,[Thanks to the fact that the Trump administrat...,Thanks to the fact that the Trump administrati...,"[[{'label': 'surprise', 'score': 0.88948446512...","[[{'label': 'LABEL_0', 'score': 0.818992555141..."
...,...,...,...,...,...,...,...,...,...
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Illinois Lottery,ORG,[The state will be suspended from participatin...,The state will be suspended from participating...,"[[{'label': 'sadness', 'score': 0.664979696273...","[[{'label': 'LABEL_2', 'score': 0.889088034629..."
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Jason Schaumburg,PERSON,[The state will be suspended from participatin...,The state will be suspended from participating...,"[[{'label': 'sadness', 'score': 0.664979696273...","[[{'label': 'LABEL_2', 'score': 0.889088034629..."
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Gianna Urgo,PERSON,"[With no money to pay road contractors, Illino...","With no money to pay road contractors, Illinoi...","[[{'label': 'sadness', 'score': 0.831223368644...","[[{'label': 'LABEL_2', 'score': 0.901882171630..."
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Medicaid,ORG,[A U.S. judge has set a June 20 deadline for I...,A U.S. judge has set a June 20 deadline for Il...,"[[{'label': 'neutral', 'score': 0.787567675113...","[[{'label': 'LABEL_2', 'score': 0.913707435131..."


In [8]:
def NRC_tagging(text):
    emotion = NRCLex(text)
    return emotion.raw_emotion_scores

NRC_tagging(train_df.joined_sentences.iloc[0])

{'sadness': 6,
 'negative': 7,
 'surprise': 2,
 'anticipation': 6,
 'positive': 14,
 'trust': 12,
 'fear': 4,
 'joy': 4}

In [9]:
train_df['NRC'] = train_df.joined_sentences.progress_apply(lambda x: NRC_tagging(x))
train_df.to_pickle('train_NER_emotion_stance.h5')


val_df['NRC'] = val_df.joined_sentences.progress_apply(lambda x: NRC_tagging(x))
val_df.to_pickle('val_NER_emotion_stance.h5')

test_df['NRC'] = test_df.joined_sentences.progress_apply(lambda x: NRC_tagging(x))
test_df.to_pickle('test_NER_emotion_stance.h5')

100%|████████████████████████████████████████████████████████████████████████| 226180/226180 [03:17<00:00, 1146.68it/s]
